# SLM: Wiki40B日本語データセットでのDiffusionモデル学習

このノートブックでは、日本語Wiki40Bデータセットを使用してWave Network言語モデルをDiffusionアプローチで学習します。

## 1. 環境セットアップ

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# プロジェクトのクローンとインストール
!git clone https://github.com/yourusername/slm_project.git
!cd slm_project && pip install -e .

In [ ]:
# 必要なライブラリのインストール
!pip install sentencepiece datasets transformers==4.31.0 accelerate==0.21.0 pywavelets==1.4.1 bitsandbytes

## 2. Wiki40B日本語データセットの前処理

In [ ]:
# 必要なライブラリをインポート
import os
from datasets import load_dataset

# Google Drive上のデータディレクトリ
data_dir = "/content/drive/MyDrive/slm/data/wiki40b_ja"
os.makedirs(data_dir, exist_ok=True)

# 最新バージョンのslmパッケージを使用するために更新
!cd /content/slm_project && pip install -e .

# toramaru-u/wiki40b-ja データセットを直接ロード (前処理済み)
dataset = load_dataset("toramaru-u/wiki40b-ja")

# データセット情報を表示
print(f"データセット構造:")
print(f"  スプリット: {list(dataset.keys())}")
for split in dataset:
    print(f"  {split}: {len(dataset[split])}サンプル")

# ローカルに保存（将来の使用のため）
dataset.save_to_disk(data_dir)
print(f"データセット保存完了。保存パス: {data_dir}")

In [ ]:
# megagonlabs/t5-base-japanese-webトークナイザーをロード
from transformers import AutoTokenizer
from slm.tokenizer import JapaneseTokenizer

tokenizer_name = "megagonlabs/t5-base-japanese-web"
print(f"トークナイザー {tokenizer_name} をロード中...")

# transformersからHuggingFaceトークナイザーをロード
hf_tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# JapaneseTokenizerラッパーに変換
jp_tokenizer = JapaneseTokenizer.from_pretrained_tokenizer(hf_tokenizer)

print(f"トークナイザーをロードしました。語彙サイズ: {len(hf_tokenizer.vocab) if hasattr(hf_tokenizer, 'vocab') else hf_tokenizer.vocab_size}")

# トークナイザーを保存（必要に応じて）
tokenizer_path = os.path.join(data_dir, "tokenizer")
os.makedirs(tokenizer_path, exist_ok=True)
hf_tokenizer.save_pretrained(tokenizer_path)
print(f"トークナイザーを保存しました: {tokenizer_path}")

In [ ]:
# トークナイザーのテスト
test_text = "これはトークナイザーのテストです。日本語Wikipediaで学習されたモデルを使います。"
print(f"テスト文: {test_text}")

# トークン化
tokens_ids = jp_tokenizer.encode(test_text)
tokens_str = hf_tokenizer.convert_ids_to_tokens(tokens_ids)
print(f"トークンID: {tokens_ids}")
print(f"トークン: {tokens_str}")

# デコード (例外処理対応)
try:
    # 新しいdecodeメソッドを試す
    decoded_text = jp_tokenizer.decode(tokens_ids, skip_special_tokens=True)
except TypeError:
    # 引数エラーが出た場合は、通常のデコードを行い、後で特殊トークンを削除
    decoded_text = jp_tokenizer.decode(tokens_ids)
    # 特殊トークン（</s>など）を削除
    decoded_text = decoded_text.replace("</s>", "")

print(f"デコード結果: {decoded_text}")

# 一致確認
if test_text == decoded_text:
    print("✓ 完全一致しました")
else:
    print("× 不一致があります")
    print(f"  元のテキスト: {test_text}")
    print(f"  デコード結果: {decoded_text}")

# 特殊トークンを含むデコード結果も表示
try:
    decoded_with_special = jp_tokenizer.decode(tokens_ids, skip_special_tokens=False)
except TypeError:
    decoded_with_special = jp_tokenizer.decode(tokens_ids)

print(f"\n特殊トークンを含むデコード結果: {decoded_with_special}")

# [MASK]トークンのIDを確認
mask_token_id = jp_tokenizer.mask_token_id
print(f"[MASK]トークンID: {mask_token_id}")

# トークナイザー情報
print(f"\nトークナイザー情報:")
print(f"  元のトークナイザー: {hf_tokenizer.__class__.__name__}")
print(f"  語彙サイズ: {len(hf_tokenizer.vocab) if hasattr(hf_tokenizer, 'vocab') else hf_tokenizer.vocab_size}")
special_tokens = {
    "MASK": hf_tokenizer.mask_token if hasattr(hf_tokenizer, 'mask_token') else None,
    "PAD": hf_tokenizer.pad_token if hasattr(hf_tokenizer, 'pad_token') else None,
    "EOS": hf_tokenizer.eos_token if hasattr(hf_tokenizer, 'eos_token') else None,
    "BOS": hf_tokenizer.bos_token if hasattr(hf_tokenizer, 'bos_token') else None
}
for name, token in special_tokens.items():
    print(f"  {name}: {token if token else 'なし'}")

## 3. モデルの学習

In [ ]:
# データセットの前処理を行い、トークン化
from datasets import load_from_disk

# すでに保存されたデータセットをロード
dataset = load_from_disk(data_dir)

# トークン化関数
def tokenize_function(examples):
    tokenized = {"input_ids": [], "attention_mask": []}
    
    for text in examples["text"]:
        # トークン化
        token_ids = jp_tokenizer.encode(text)
        
        # 最大長に切り詰め (512トークン)
        if len(token_ids) > 512:
            token_ids = token_ids[:512]
        
        # 注意マスクを作成（すべて1）
        attn_mask = [1] * len(token_ids)
        
        tokenized["input_ids"].append(token_ids)
        tokenized["attention_mask"].append(attn_mask)
    
    return tokenized

# データセットをトークン化
print("データセットをトークン化中...")
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=1000,
    remove_columns=["text"]
)

# 検証セットのサイズを制限（メモリ節約のため）
if "validation" in tokenized_datasets:
    tokenized_datasets["validation"] = tokenized_datasets["validation"].select(range(min(len(tokenized_datasets["validation"]), 1000)))

print("トークン化済みデータセット情報:")
for split in tokenized_datasets:
    print(f"  {split}: {len(tokenized_datasets[split])}サンプル")

# トークン化済みデータセットを保存
tokenized_path = os.path.join(data_dir, "tokenized")
tokenized_datasets.save_to_disk(tokenized_path)
print(f"トークン化済みデータセットを保存しました: {tokenized_path}")

# 新しいDiffusionモデル学習スクリプトを実行
!cd /content/slm_project && python slm/train_wiki40b_ja_diffusion_megagon.py \
    --use_local_dataset \
    --local_data_dir="{tokenized_path}" \
    --output_dir="/content/drive/MyDrive/slm/outputs" \
    --tokenizer_name="megagonlabs/t5-base-japanese-web" \
    --hidden_size=1024 \
    --num_layers=3 \
    --max_seq_len=512 \
    --batch_size=8 \
    --epochs=3 \
    --learning_rate=2e-5

## 4. 学習結果の確認

In [ ]:
import os
import torch
from slm.modules.wave_network import WaveNetworkLM
from slm.config import ModelConfig
from slm.wiki40b_ja_dataset import load_tokenizer

# モデルチェックポイントパス
output_dir = "/content/drive/MyDrive/slm/outputs"
run_dirs = [d for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d)) and d.startswith("wiki40b_ja_diffusion")]
run_dirs.sort()
latest_run = run_dirs[-1] if run_dirs else None

if latest_run:
    checkpoint_dir = os.path.join(output_dir, latest_run, "checkpoints")
    checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.endswith(".pt")]
    checkpoint_files.sort()
    latest_checkpoint = os.path.join(checkpoint_dir, checkpoint_files[-1]) if checkpoint_files else None
    
    print(f"最新の実行: {latest_run}")
    print(f"利用可能なチェックポイント: {checkpoint_files}")
    print(f"最新のチェックポイント: {latest_checkpoint}")
else:
    print("学習済みのモデルが見つかりません")

In [ ]:
# 学習済みモデルのロード
if 'latest_checkpoint' in locals() and latest_checkpoint:
    from transformers import AutoTokenizer
    from slm.tokenizer import JapaneseTokenizer
    
    # トークナイザーのロード
    tokenizer_name = "megagonlabs/t5-base-japanese-web"
    hf_tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    tokenizer = JapaneseTokenizer.from_pretrained_tokenizer(hf_tokenizer)
    
    # チェックポイントのロード
    checkpoint = torch.load(latest_checkpoint, map_location='cpu')
    model_config = checkpoint["model_config"]
    
    # トークナイザーをモデルに設定
    model_config.set_tokenizer(tokenizer)
    
    # モデルのインスタンス化と重みのロード
    model = WaveNetworkLM(model_config)
    model.load_state_dict(checkpoint["model_state_dict"])
    
    print(f"モデルを正常にロードしました。パラメータ数: {sum(p.numel() for p in model.parameters()):,}")
else:
    print("学習済みモデルがロードできませんでした")

## 5. 簡単な推論テスト

In [ ]:
# 簡単な推論テスト（モデルがロードされている場合）
if 'model' in locals() and 'tokenizer' in locals():
    from slm.diffusion import SimpleTextDiffusion
    import torch
    
    # デバイスの設定
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    # マスクトークンIDの取得
    mask_token_id = tokenizer.piece_to_id("[MASK]")
    
    # Diffusionモデルのインスタンス化
    diffuser = SimpleTextDiffusion(
        timesteps=20,
        mask_token_id=mask_token_id,
        vocab_size=tokenizer.get_piece_size()
    ).to(device)
    
    # テスト文の用意
    test_text = "日本は四季折々の自然が美しい国です。"
    print(f"元のテキスト: {test_text}")
    
    # トークン化
    tokens = tokenizer.encode(test_text, out_type=int)
    token_tensor = torch.tensor([tokens], device=device)
    
    # 完全にノイズを加える（最大タイムステップ)
    t = torch.tensor([diffuser.timesteps - 1], device=device)
    noisy_tokens, _ = diffuser(token_tensor, t)
    
    # ノイズを加えたテキストの表示
    noisy_text = tokenizer.decode(noisy_tokens[0].cpu().tolist())
    print(f"ノイズを加えたテキスト: {noisy_text}")
    
    # 逐次的にノイズを除去する簡易版デノイズプロセス
    with torch.no_grad():
        current_tokens = noisy_tokens.clone()
        
        for timestep in reversed(range(diffuser.timesteps)):
            # マスクされた位置を見つける
            mask_positions = (current_tokens == mask_token_id)
            
            if not mask_positions.any():
                break
                
            # モデルの予測を取得
            logits = model(current_tokens)
            
            # マスクされた位置でのトップk予測を取得
            k = 5
            topk_probs, topk_indices = torch.topk(logits.softmax(dim=-1), k, dim=-1)
            
            # マスクごとにトップ1の予測で置き換え
            for i in range(current_tokens.size(0)):
                for j in range(current_tokens.size(1)):
                    if mask_positions[i, j]:
                        # ここではトップ1の予測を使用
                        current_tokens[i, j] = topk_indices[i, j, 0]
                        
            # 結果を表示
            if timestep % 5 == 0 or timestep == 0:
                current_text = tokenizer.decode(current_tokens[0].cpu().tolist())
                print(f"タイムステップ {timestep} での復元: {current_text}")
        
        final_text = tokenizer.decode(current_tokens[0].cpu().tolist())
        print(f"\n最終的な復元テキスト: {final_text}")
else:
    print("モデルがロードされていないため、推論テストを実行できません")